In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# Część 2

1. Zmierzyć fotoprąd wytwarzany w ogniwie w funkcji długości fali w zakresie 4000-
1200 nm. Wartości długości fali dobrać tak, aby punkty pomiarowe były równomiernie
i wystarczająco gęsto (nie rzadziej niż co 0,05 eV) rozłożone w skali energii.
2. Wykorzystując plik photon flux zawierający rozkład spektralny światła padającego
na próbkę, wykonać wykres rozkładu fotoprądu w funkcji energii padających fotonów
znormalizowany na jeden padający foton (ponieważ nie znamy dokładnej liczby
fotonów padających na strukturę, możemy jedynie określić ten rozkład w jednostkach
dowolnych)
3. Wyznaczyć wartość przerwy energetycznej absorbera posługując się metodą „w
połowie krawędzi”.
4. Wyznaczyć energię odpowiadającą wysokoenergetycznej krawędzi rozkładu. Czemu
on odpowiada?
5. Porównać wartość Voc(T) dla T->0 z wartością przerwy energetycznej otrzymaną
z ekstrapolacji zależności Voc(T) do 0oK, wyciągnąć wnioski.